In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv",
                             nrows=int(0.5 * len(pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"))))
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [124]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [125]:
# Determine the number of unique values in each column.
for x in application_df.columns:
    print(x, len(application_df[x].unique()))

APPLICATION_TYPE 14
AFFILIATION 6
CLASSIFICATION 47
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 4158
IS_SUCCESSFUL 2


In [126]:
# Look at APPLICATION_TYPE value counts for binning
counts = application_df['APPLICATION_TYPE'].value_counts()
counts

T3     13822
T6       664
T4       592
T19      459
T8       433
T10      401
T7       375
T5       264
T9        84
T13       33
T12       11
T2         8
T29        2
T25        1
Name: APPLICATION_TYPE, dtype: int64

In [127]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts[counts<5].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       13822
T6         664
T4         592
T19        459
T8         433
T10        401
T7         375
T5         264
T9          84
T13         33
T12         11
T2           8
Other        3
Name: APPLICATION_TYPE, dtype: int64

In [128]:
# Look at CLASSIFICATION value counts for binning
bincounts = application_df['CLASSIFICATION'].value_counts()

In [129]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classcounts = bincounts[bincounts>100]
classcounts

C1000    8885
C2000    2982
C1200    2396
C2100    1136
C3000     580
C7000     420
C1700     174
C4000     117
Name: CLASSIFICATION, dtype: int64

In [130]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list (bincounts[bincounts<95].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    8885
C2000    2982
C1200    2396
C2100    1136
C3000     580
Other     459
C7000     420
C1700     174
C4000     117
Name: CLASSIFICATION, dtype: int64

In [131]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [132]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values

X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 50)

In [133]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train and Evaluate the Model

In [134]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=7
hidden_nodes_layer2=14
hidden_nodes_layer3=21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 7)                 357       
                                                                 
 dense_22 (Dense)            (None, 14)                112       
                                                                 
 dense_23 (Dense)            (None, 1)                 15        
                                                                 
Total params: 484
Trainable params: 484
Non-trainable params: 0
_________________________________________________________________


In [135]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [137]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
342/342 [==============================] - 2s 3ms/step - loss: 0.6505 - accuracy: 0.6423 - val_loss: 0.5818 - val_accuracy: 0.7181
Epoch 2/100
342/342 [==============================] - 1s 2ms/step - loss: 0.5799 - accuracy: 0.7193 - val_loss: 0.5546 - val_accuracy: 0.7482
Epoch 3/100
342/342 [==============================] - 1s 3ms/step - loss: 0.5638 - accuracy: 0.7279 - val_loss: 0.5466 - val_accuracy: 0.7523
Epoch 4/100
342/342 [==============================] - 1s 3ms/step - loss: 0.5571 - accuracy: 0.7331 - val_loss: 0.5417 - val_accuracy: 0.7523
Epoch 5/100
342/342 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7331 - val_loss: 0.5383 - val_accuracy: 0.7528
Epoch 6/100
342/342 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7402 - val_loss: 0.5358 - val_accuracy: 0.7596
Epoch 7/100
342/342 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7415 - val_loss: 0.5324 - val_accuracy: 0.7591

In [138]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

134/134 - 0s - loss: 0.5373 - accuracy: 0.7449 - 210ms/epoch - 2ms/step
Loss: 0.5373286008834839, Accuracy: 0.7448694109916687


# Optimization 2

In [6]:
# Drop the non-beneficial ID columns, 'EIN'.
application_df = application_df.drop(columns = ['EIN'])

In [7]:
# Determine the number of unique values in each column.
for x in application_df.columns:
    print(x, len(application_df[x].unique()))

NAME 10554
APPLICATION_TYPE 14
AFFILIATION 6
CLASSIFICATION 47
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 4158
IS_SUCCESSFUL 2


In [8]:
# Look at NAME value counts for binning
counts2 = application_df['NAME'].value_counts()
counts2

PARENT BOOSTER USA INC                                          746
UNITED STATES BOWLING CONGRESS INC                              489
ALPHA PHI SIGMA                                                 294
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS               245
TOPS CLUB INC                                                   236
                                                               ... 
WILDLIFE REHABILITATION CENTER OF NORTHERN UTAH                   1
HOUSTON FAITH FAMILY CHURCH INC                                   1
SOUTHERN SPORTSMEN FOR YOUTH & PHYSICALLY CHALLENGED HUNTERS      1
FLORIDIANS FOR TAX FAIRNESS INC                                   1
CAPITOL HILL HIGH SCHOOL ALUMNI HALL OF FAME ASSOCIATION          1
Name: NAME, Length: 10554, dtype: int64

In [10]:
# Choose a cutoff value and create a list of names to be replaced
# use the variable name `names_to_replace`
names_to_replace = list(counts2[counts2<5].index)
names_to_replace

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                                              10749
PARENT BOOSTER USA INC                                               746
UNITED STATES BOWLING CONGRESS INC                                   489
ALPHA PHI SIGMA                                                      294
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                    245
                                                                   ...  
JACKSONVILLE DISTRICT COUNCIL SOCIETY OF ST VINCENT DE PAUL INC        5
ZETA PHI BETA SORORITY INC                                             5
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS                           5
CALIFORNIA WOMENS CHORUS                                               5
NATIONAL COUNCIL FOR GEOCOSMIC RESEARCH INC                            5
Name: NAME, Length: 231, dtype: int64

In [12]:
# Look at APPLICATION TYPE value counts for binning
bincounts = application_df['APPLICATION_TYPE'].value_counts()
bincounts

T3     13822
T6       664
T4       592
T19      459
T8       433
T10      401
T7       375
T5       264
T9        84
T13       33
T12       11
T2         8
T29        2
T25        1
Name: APPLICATION_TYPE, dtype: int64

In [13]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list (bincounts[bincounts<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       13822
Other     2071
T6         664
T4         592
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# Look at CLASSIFICATION value counts for binning
bincounts2 = application_df['CLASSIFICATION'].value_counts()

In [15]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classcounts = bincounts[bincounts>1]
classcounts

T3     13822
T6       664
T4       592
T19      459
T8       433
T10      401
T7       375
T5       264
T9        84
T13       33
T12       11
T2         8
T29        2
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list (bincounts2[bincounts2<100].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    8885
C2000    2982
C1200    2396
C2100    1136
C3000     580
Other     459
C7000     420
C1700     174
C4000     117
Name: CLASSIFICATION, dtype: int64

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI OMEGA,NAME_ALPHA PHI SIGMA,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values

X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 50)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train and Evaluate the Model

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=7
hidden_nodes_layer2=14
hidden_nodes_layer3=21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 1911      
                                                                 
 dense_1 (Dense)             (None, 14)                112       
                                                                 
 dense_2 (Dense)             (None, 1)                 15        
                                                                 
Total params: 2,038
Trainable params: 2,038
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
342/342 [==============================] - 4s 7ms/step - loss: 0.5698 - accuracy: 0.7188 - val_loss: 0.4818 - val_accuracy: 0.7886
Epoch 2/100
342/342 [==============================] - 1s 3ms/step - loss: 0.4808 - accuracy: 0.7764 - val_loss: 0.4450 - val_accuracy: 0.8005
Epoch 3/100
342/342 [==============================] - 1s 2ms/step - loss: 0.4569 - accuracy: 0.7849 - val_loss: 0.4289 - val_accuracy: 0.8010
Epoch 4/100
342/342 [==============================] - 1s 2ms/step - loss: 0.4441 - accuracy: 0.7915 - val_loss: 0.4169 - val_accuracy: 0.8088
Epoch 5/100
342/342 [==============================] - 1s 2ms/step - loss: 0.4369 - accuracy: 0.7932 - val_loss: 0.4164 - val_accuracy: 0.8078
Epoch 6/100
342/342 [==============================] - 1s 2ms/step - loss: 0.4333 - accuracy: 0.7938 - val_loss: 0.4155 - val_accuracy: 0.8119
Epoch 7/100
342/342 [==============================] - 1s 2ms/step - loss: 0.4318 - accuracy: 0.7948 - val_loss: 0.4134 - val_accuracy: 0.8119

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

134/134 - 0s - loss: 0.4501 - accuracy: 0.7999 - 181ms/epoch - 1ms/step
Loss: 0.45012167096138, Accuracy: 0.7999067306518555


In [24]:
# Export our model to HDF5 file
from google.colab import files

nn.save('/content/AlphabetSoupCharity_Optimization.h5')
files.download('/content/AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>